# 使用Fluid构建自定义的机器学习训练流水线

机器学习训练流水线往往伴随着大量数据处理和消费的流程，包括数据下载、数据预处理、模型训练以及模型Checkpoint的归档等多个步骤。这些串联的步骤可被视作数据在不同存储系统和计算系统间的流转过程，即上一步骤的输出文件作为下一步骤的输入文件，从而形成一条完整的数据流。然而，不同步骤产出的数据往往有不同的生命周期，例如：下载得到的原始数据集和训练得到的模型Checkpoint需要持久化地存储在大容量存储中，而预处理后的数据在模型训练运行成功后即可删除，无需持久化保存。另外，不同步骤对于数据的消费和处理过程有着不同的性能要求，数据预处理步骤和模型训练步骤相比其他步骤有着更高的数据I/O吞吐需求，以此确保计算过程尽快完成，提升运行效率。

Fluid是一个开源的Kubernetes原生的分布式数据集编排和加速引擎，Fluid通过在大容量的云存储系统（e.g. S3, HDFS等)之上添加缓存系统层（Alluxio, JuiceFS等），分别满足机器学习流水线各步骤的数据持久化和性能需求。Fluid支持定义数据下载、数据迁移、数据预处理、数据消费（例如模型训练）等多个步骤，并支持将数据流转流程串联，构建自定义的机器学习流水线。

接下来展示如何使用Fluid Python SDK定义并提交示例机器学习训练流水线任务。本示例包括如下几步：
1. 原始数据集迁移：从阿里云OSS对象存储系统中将MNIST原始数据集gz压缩文件迁移到JuiceFS缓存系统。
2. 数据集预处理：将MNIST原始数据集解压
3. 模型训练：使用Kubeflow/Arena提交模型训练任务。模型训练任务基于MNIST解压后的数据集文件，训练CNN模型，训练完成后CNN模型Checkpoint写入到JuiceFS缓存系统中
4. 模型归档：模型Checkpoint文件迁移到阿里云OSS对象存储系统中，归档训练好的模型。

## 前提条件
- 可连通的Kubernetes集群，集群中已安装Fluid
- 配置缓存系统所需的后端存储系统、访问凭证等信息。本示例中使用JuiceFS作为缓存系统，JuiceFS元信息服务为redis，对象存储系统为minio。redis与minio均在Kubernetes集群中运行。
- [MNIST原始数据集](http://yann.lecun.com/exdb/mnist/)，下载并存储到阿里云OSS对象存储中，本示例中假设该路径为`oss://<OSS_BUCKET>/mnist`
- 安装Fluid Python SDK

In [1]:
# Install Fluid Python SDK
# !pip config set global.index-url https://pypi.tuna.tsinghua.edu.cn/simple
%pip install git+https://github.com/fluid-cloudnative/fluid-client-python.git
%pip install kubernetes

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
  Cloning https://github.com/fluid-cloudnative/fluid-client-python.git to /private/var/folders/np/hp7jhh3n7fz_wtbg5rxyk0g00000gp/T/pip-req-build-7ilvvego
  Running command git clone --filter=blob:none --quiet https://github.com/fluid-cloudnative/fluid-client-python.git /private/var/folders/np/hp7jhh3n7fz_wtbg5rxyk0g00000gp/T/pip-req-build-7ilvvego
  Resolved https://github.com/fluid-cloudnative/fluid-client-python.git to commit f93c6b0e833d9073b726db891216175a2a67e6b6
  Preparing metadata (setup.py) ... done
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Note: you may need to restart the kernel to use updated packages.


In [2]:
import fluid
fluid.__version__

'0.1'

## 准备Fluid Dataset

In [3]:
# Initialize fluid client
from fluid import constants
from fluid import models

from kubernetes import client

client_config = fluid.ClientConfig()
fluid_client = fluid.FluidClient(client_config)

In [4]:
# Setting fluidsdk logger level to DEBUG for detailed messages
import logging
import sys
logger = logging.getLogger("fluidsdk")
stream_handler = logging.StreamHandler(sys.stdout)
stream_handler.setFormatter(logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s'))
logger.addHandler(stream_handler)
logger.setLevel(logging.DEBUG)

In [5]:
# Create Fluid Dataset
secret_name="jfs-secret"
options = {
    "bucket": "http://minio:9000/minio/test",
    "storage": "minio"
}
cred_secret_options = {
    "access-key": "access-key", # <encrypt_option_key>: <key_reference_in_secret> 
    "secret-key": "secret-key",
    "metaurl": "metaurl"
}
fluid_client.create_dataset(dataset_name="mydataset", mount_name="minio", mount_point="juicefs:///", mode="ReadWrite", options=options, cred_secret_name=secret_name, cred_secret_options=cred_secret_options)

2024-01-15 20:15:37,006 - fluidsdk - DEBUG - Dataset "default/mydataset" created


In [6]:
# Bind a runtime to the created Dataset
dataset = fluid_client.get_dataset(dataset_name="mydataset")
dataset.bind_runtime(runtime_type=constants.JUICEFS_RUNTIME_KIND, replicas=2, cache_capacity_GiB=10, cache_medium="MEM", wait=True)

## 构建机器学习训练流水线

首先，获取刚才创建的Fluid Dataset实例

In [7]:
dataset.report_status("binding_status")

{'phase': 'Bound',
 'runtimes': [{'category': 'Accelerate',
   'master_replicas': 2,
   'name': 'mydataset',
   'namespace': 'default',
   'type': 'juicefs'}]}

Fluid Dataset实例支持一系列数据处理操作，包括数据迁移、数据缓存预热和数据处理，按照机器学习训练流水线的步骤，首先将阿里云OSS对象存储系统中的MNIST原始数据集迁移到`mydataset`中。

In [8]:
def get_oss_encrypt_options(secret_name):
    # Replace the following with your own secret references
    encrypt_options = []
    encrypt_options.append(models.EncryptOption(
        name="access-key",
        value_from=models.EncryptOptionSource(
            secret_key_ref=models.SecretKeySelector(
                name=secret_name,
                key="access-key"
            )
        )
    ))
    encrypt_options.append(models.EncryptOption(
        name="secret-key",
        value_from=models.EncryptOptionSource(
            secret_key_ref=models.SecretKeySelector(
                name=secret_name,
                key="secret-key"
            )
        )
    ))
    return encrypt_options

flow = dataset.migrate(path="/MNIST/raw/", migrate_direction=constants.DATA_MIGRATE_DIRECTION_FROM,
                    external_storage=models.ExternalStorage(
                        uri="oss://<OSS_BUCKET>.<OSS_ENDPOINT>.aliyuncs.com/mnist/",
                        encrypt_options=get_oss_encrypt_options("oss-access-creds")))

上述代码片段中，我们通过`ds.migrate()`创建了一条Fluid数据流（DataFlow），该数据流采用一种懒惰执行(Lazy Execute)的方式，因此在实际执行前我们可以在该数据流后追加其他数据操作。例如，新增数据处理操作，解压迁移的MNIST原始数据集，并将解压结果存储到Dataset `mydataset`中。

In [9]:
decompress_mnist_script = """
#!/bin/bash
set -ex

cd /data/mnist

gzip -d --keep t10k-images-idx3-ubyte.gz
gzip -d --keep t10k-labels-idx1-ubyte.gz
gzip -d --keep train-images-idx3-ubyte.gz
gzip -d --keep train-labels-idx1-ubyte.gz
"""

flow = flow.process(dataset_mountpath="/data/mnist/", sub_path="MNIST/raw/", processor=models.Processor(
            script=models.ScriptProcessor(
                command=["bash"],
                source=decompress_mnist_script,
                image="debian",
                image_tag="buster")))

接着，我们显式地对MNIST数据集执行预热，即将解压后的MNIST数据集提前加载到缓存系统中，以增加后续模型训练时的数据读取效率。

In [10]:
flow = flow.preload(target_path="/MNIST/raw")

定义完数据缓存预热的数据流步骤后，使用数据处理操作定义一个新的数据流步骤，在这个步骤中我们使用Arena命令行工具提交一个PyTorch训练任务，训练任务的镜像中包含了PyTorch的基础环境以及PyTorch官方的MNIST训练代码示例。该训练任务读取MNIST数据集并训练一个CNN模型，并会将CNN模型的Checkpoint文件写入到缓存系统中。

In [11]:
def get_submit_job_script(dataset_name, job_image, command):
    return f"""
    #!/bin/bash
    set -ex
    arena submit pytorch \
        --name=mnist-pytorch \
        --gpus=1 \
        --workers=1 \
        --image={job_image} \
        --data={dataset_name}:/data \
        {command}
        
    while true; do
        sleep 5
        status=$(arena get mnist-pytorch -o json | jq .status | tr -d '"')
        if [ "$status" == "SUCCEEDED" ]; then
            break
        fi
    done
    """

flow = flow.process(dataset_mountpath="/data/mnist", sub_path="MNIST/raw", processor=models.Processor(
            service_account_name="fluid-demo",
            script=models.ScriptProcessor(
                command=["bash"],
                source=get_submit_job_script("mydataset", "cloudnative4ai/torch-samples:cuda-mnist", "'python /workspace/main.py --epochs 2 --data /data --save-model --save-model-path /data/ckpt'"),
                image="registry.cn-beijing.aliyuncs.com/fluid-namespace/python-arena",
                image_tag="3.7-0.9.11-ce87d10-01101736")))

在上面定义的数据处理步骤中，需要额外指定一个`service_account_name`字段，对应的ServiceAccount应当具有Arena PyTorchJob资源的创建和状态查看权限。另外，我们设置PyTorch模型训练任务的`--save-model-path`参数，指定程序将训练完成的CNN模型储存到Dataset `mydataset`中（在Arena任务提交脚本中指定了将`mydataset`挂载到模型训练任务Pod的`/data`目录）。

最后，我们定义最后一个数据流步骤——将模型Checkpoint文件迁移到持久化存储的阿里云OSS对象存储系统中，归档训练好的CNN模型。

In [12]:
flow = flow.migrate(path="/ckpt/mnist_cnn.pt", migrate_direction=constants.DATA_MIGRATE_DIRECTION_TO,
                    external_storage=models.ExternalStorage(
                        uri="oss://<OSS_BUCKET>.<OSS_ENDPOINT>/mnist_ckpt/mnist_cnn.pt",
                        encrypt_options=get_oss_encrypt_options("oss-access-creds")))

完成数据流定义后，通过以下命令执行完整的数据流，每次执行需要指定一个RUN ID，并返回一次数据流RUN的实例对象，使用run.wait()可以等待阻塞等待流水线运行完成。

In [13]:
run = flow.run(run_id="torch-mnist-training")
run.wait()

2024-01-15 20:17:14,765 - fluidsdk - INFO - DataMigrate torch-mnist-training-step1 completed
2024-01-15 20:17:38,962 - fluidsdk - INFO - DataProcess torch-mnist-training-step2 completed
2024-01-15 20:18:03,143 - fluidsdk - INFO - DataLoad torch-mnist-training-step3 completed
2024-01-15 20:18:48,476 - fluidsdk - INFO - DataProcess torch-mnist-training-step4 completed
2024-01-15 20:19:12,655 - fluidsdk - INFO - DataMigrate torch-mnist-training-step5 completed


## 清理Fluid Dataset
机器学习训练流水线执行完成后，可将缓存系统缩容或删除，来清理占用的缓存资源。

In [ ]:
dataset.clean_up(wait=True)